In [3]:
import pandas as pd
from pandas import DataFrame
import numpy as np

from os import listdir
from os.path import isfile, join

from PIL import Image

from math import pi
from numpy import arctan
from numpy import sinh

In [4]:
#Testando a criação de um arquivo csv
'''

teste = [(32.308027389210054, -83.82843017578125), (33.308027389210054, -83.82843017578125), (52.308027389210054, -3.82843017578125)]

data = DataFrame(teste)

data.to_csv("output/teste.csv")

data
'''

'\n\nteste = [(32.308027389210054, -83.82843017578125), (33.308027389210054, -83.82843017578125), (52.308027389210054, -3.82843017578125)]\n\ndata = DataFrame(teste)\n\ndata.to_csv("output/teste.csv")\n\ndata\n'

In [5]:
#Testando como criar uma lista com os nomes dos arquivos de um diretório
'''

path = 'input'
files = [f for f in listdir(path) if isfile(join(path, f))]
print(files)
'''

"\n\npath = 'input'\nfiles = [f for f in listdir(path) if isfile(join(path, f))]\nprint(files)\n"

In [6]:
#Teste de conversão de uma imagem para um arquivo csv.
''' 


path = 'input'

#for img in listdir(path):
#    if isfile(join(path, img)):
#        im = Image.open(join(path, img)).convert("L")
#        im = im.crop((256, 0, 512, 256))
#        im.show()
im = Image.open("input/t_16780_24825_16.png").convert("L")
im = im.crop((256, 0, 512, 256))
im.show()



pixels = list(im.getdata())

p = np.array(pixels)
p.resize((256, 256))
p = DataFrame(p)

p.to_csv('teste.csv')

'''

' \n\n\npath = \'input\'\n\n#for img in listdir(path):\n#    if isfile(join(path, img)):\n#        im = Image.open(join(path, img)).convert("L")\n#        im = im.crop((256, 0, 512, 256))\n#        im.show()\nim = Image.open("input/t_16780_24825_16.png").convert("L")\nim = im.crop((256, 0, 512, 256))\nim.show()\n\n\n\npixels = list(im.getdata())\n\np = np.array(pixels)\np.resize((256, 256))\np = DataFrame(p)\n\np.to_csv(\'teste.csv\')\n\n'

In [7]:
# Funções para converter a posição de um pixel em um tile para um par de coordenadas.

def get_lat(x, y, z):
  n = 2 ** z

  min_rad = arctan(sinh(pi * (1 - 2 * y / n)))
  min_deg = min_rad * 180.0 / pi

  max_rad = arctan(sinh(pi * (1 - 2 * (y+1) / n)))
  max_deg = max_rad * 180.0 / pi

  return min_deg, max_deg


def get_lon(x, y, z):
  n = 2 ** z
  min_deg = x / n * 360.0 - 180.0
  max_deg = (x+1) / n * 360.0 - 180.0
  return min_deg, max_deg


def pixToCoord(x, y, z, t_height, t_width, px, py):

  [min_lat, max_lat] = get_lat(x,y,z)
  [min_lon, max_lon] = get_lon(x,y,z)
  
  lat = min_lat + py*((max_lat - min_lat)/t_height)
  lon = min_lon + px*((max_lon - min_lon)/t_width)
  
  coord = (lat, lon)
  

  return coord



In [8]:
#Função de conversão de uma imagem para um DataFrame.

def imgToDf(path):
    im = Image.open(path).convert("L")
    im = im.crop((256, 0, 512, 256))

    pixels = list(im.getdata())

    p = np.array(pixels)
    p.resize((256, 256))
    p = DataFrame(p)

    
    return p

In [9]:
# Testando iteração por toda um diretório, gerando um diretório de saida com o arquivo csv correspondente a cada imagem.

'''
path = 'input'
listCoord = []

for img in listdir(path):
    if isfile(join(path, img)):
        

        imgOutPath = img
        imgOutPath = imgOutPath.replace('.png', '.csv')
        
        
        imgCopy = img.replace('t', '')
        imgCopy = imgCopy.replace('_', '')
        imgCopy = imgCopy.replace('.png', '')
        x = int( imgCopy[:5] )
        y = int( imgCopy[5:10] )
        z = int( imgCopy[10:12] )

        imgPix = imgToDf(join(path, img))
        
        rows, cols = imgPix.shape

        data = []

        for i in range(rows):
            for j in range(cols):
                if imgPix.iloc[i, j] != 0:
                    imgPix.iloc[i, j] = 255

                elif imgPix.iloc[i, j] == 0:
                    # j=px, i=py
                    #data.append( pixToCoord(x, y, z, 256, 256, j, i) )
                    pass


        #dataF = DataFrame(data)
        
        #dataF.to_csv("output/coord_" + imgOutPath)

        imgPix.to_csv("output/" +imgOutPath)
        
        
'''
        

'\npath = \'input\'\nlistCoord = []\n\nfor img in listdir(path):\n    if isfile(join(path, img)):\n        \n\n        imgOutPath = img\n        imgOutPath = imgOutPath.replace(\'.png\', \'.csv\')\n        \n        \n        imgCopy = img.replace(\'t\', \'\')\n        imgCopy = imgCopy.replace(\'_\', \'\')\n        imgCopy = imgCopy.replace(\'.png\', \'\')\n        x = int( imgCopy[:5] )\n        y = int( imgCopy[5:10] )\n        z = int( imgCopy[10:12] )\n\n        imgPix = imgToDf(join(path, img))\n        \n        rows, cols = imgPix.shape\n\n        data = []\n\n        for i in range(rows):\n            for j in range(cols):\n                if imgPix.iloc[i, j] != 0:\n                    imgPix.iloc[i, j] = 255\n\n                elif imgPix.iloc[i, j] == 0:\n                    # j=px, i=py\n                    #data.append( pixToCoord(x, y, z, 256, 256, j, i) )\n                    pass\n\n\n        #dataF = DataFrame(data)\n        \n        #dataF.to_csv("output/coord_" + i

In [10]:
# Função de busca em largura

import random


def findLine(image):
    img = image
    maxSort = (256 * 256)

    *numSorts, = range(0, 256)
    random.shuffle(numSorts)
    *numSorts2, = range(0, 256)
    random.shuffle(numSorts2)

    
    for i in range(0, maxSort):

        if i > 256:
            *numSorts, = range(0, 256)
            random.shuffle(numSorts)
            *numSorts2, = range(0, 256)
            random.shuffle(numSorts2)
            
        
        pxSorted = numSorts[i % 256]
        pySorted = numSorts2[i % 256]
        
        i = pySorted
        j = pxSorted
        
        if img[i, j] == 0:
            img = colorFrontier(img, j, i, 1)

    img, line = getLines(img)
    
                
    return img, line

'''

def findLine(img):

    rows, cols = img.shape

    for i in range(cols):
        for j in range(rows):

            if img[j, i] == 0:
        
                img = colorFrontier(img, j, i, 1)

    
    img, line = getLines(img, 10)
    
                
    return img, line

'''


'\n\ndef findLine(img):\n\n    rows, cols = img.shape\n\n    for i in range(cols):\n        for j in range(rows):\n\n            if img[j, i] == 0:\n        \n                img = colorFrontier(img, j, i, 1)\n\n    \n    img, line = getLines(img, 10)\n    \n                \n    return img, line\n\n'

In [11]:
# Função de busca em largura

def colorFrontier(img, px, py , state):
    visitados = set()
    prox = []
    prox.append((px, py, state))
        
    while len(prox) != 0:
        
        px, py, state = prox.pop(0)

        atual = (px, py)
        if atual not in visitados:
            img[px, py] = state

            state = state + 1
            pxMin = px - 1
            pxMax = px + 1
            pyMin = py - 1
            pyMax = py + 1
            
            if(pxMin >= 0 and img[pxMin, py] == 0):
                    prox.append( (pxMin, py, state) )

            if(pxMax < img.shape[0] and img[pxMax, py] == 0):
                    prox.append( (pxMax, py, state) )

            if(pyMin >= 0 and img[px, pyMin] == 0):
                    prox.append( (px, pyMin, state) )

            if(pyMax < img.shape[1] and img[px, pyMax] == 0):
                    prox.append( (px, pyMax, state) )
                    
                    
                   
            if(pyMax < img.shape[1] and pxMax < img.shape[0] and img[pxMax, pyMax] == 0):
                    prox.append( (pxMax, pyMax, state) )
                    
            if(pyMin >= 0 and pxMax < img.shape[0] and img[pxMax, pyMin] == 0):
                    prox.append( (pxMax, pyMin, state) )
                    
            if(pyMin >= 0 and pxMin >= 0 and img[pxMin, pyMin] == 0):
                    prox.append( (pxMin, pyMin, state) )
                    
            if(pyMax < img.shape[1] and pxMin >= 0 and img[pxMin, pyMax] == 0):
                    prox.append( (pxMin, pyMax, state) )
            
             
                
            visitados.add(atual)
            
    return img

In [12]:
# Função de busca em largura

def getLine(img):
    line = []
    maior = -1 

    rows, cols, channels = img.shape

    for i in range(cols):
        for j in range(rows):
            if img[j, i] > maior:
                maior = img[j, i]
                px = i
                py = j

    if maior <= 1:
        return -1

    line.append((maior, px, py))

    
    while maior > 0:

        pyMin = py - 1
        pyMax = py + 1
        pxMin = px - 1
        pxMax = px + 1

        
        if pyMin >= 0 and (img[pyMin, px] == (maior - 1) or  img[pyMin, px] == -2):
            maior = (maior - 1)
            if img[pyMin, px] == -2:
                maior = -2
                
            line.append((maior, px, pyMin))
            py = pyMin
            continue

        if pyMax < img.shape[0] and (img[pyMax, px] == (maior - 1) or  img[pyMax, px] == -2) :
            maior = (maior - 1)
            if img[pyMax, px] == -2:
                maior = -2

            line.append((maior, px, pyMax))
            py = pyMax
            continue

        if pxMin >= 0 and (img[py, pxMin] == (maior - 1) or  img[py, pxMin] == -2):
            maior = (maior - 1)
            if img[py, pxMin] == -2:
                maior = -2

            line.append((maior, pxMin, py))
            px = pxMin
            continue

        if pxMax < img.shape[1] and (img[py, pxMax] == (maior - 1) or  img[py, pxMax] == -2):
            maior = (maior - 1)
            if img[py, pxMax] == -2:
                maior = -2

            line.append((maior, pxMax, py))
            px = pxMax
            continue
            
            
            
        if pyMax < img.shape[1] and pxMax < img.shape[0] and (img[pyMax, pxMax] == (maior - 1) or  img[pyMax, pxMax] == -2):
            maior = (maior - 1)
            if img[pyMax, pxMax] == -2:
                maior = -2
                
            line.append((maior, pxMax, pyMax))
            py = pyMax
            px = pxMax
            continue

        if pyMin >= 0 and pxMax < img.shape[0] and (img[pyMin, pxMax] == (maior - 1) or  img[pyMin, pxMax] == -2) :
            maior = (maior - 1)
            if img[pyMin, pxMax] == -2:
                maior = -2

            line.append((maior, pxMax, pyMin))
            py = pyMin
            px = pxMax
            continue

        if pyMin >= 0 and pxMin >= 0 and (img[pyMin, pxMin] == (maior - 1) or  img[pyMin, pxMin] == -2):
            maior = (maior - 1)
            if img[pyMin, pxMin] == -2:
                maior = -2

            line.append((maior, pxMin, pyMin))
            py = pyMin
            px = pxMin
            continue

        if pyMax < img.shape[1] and pxMin >= 0 and (img[pyMax, pxMin] == (maior - 1) or  img[pyMax, pxMin] == -2):
            maior = (maior - 1)
            if img[pyMax, pxMin] == -2:
                maior = -2

            line.append((maior, pxMin, pyMax))
            py = pyMax
            px = pxMin
            continue
        
        

        break

    
    return line






In [13]:

def getLines(img, tamMinLines=10):
    lines = []
    
    while True:
        line = getLine(img)

        if line == -1:
            break 
        
        if len(line) > tamMinLines:
            lines.append( line )

        for coord in line:
            valor, px, py = coord
            if valor >= 2:
                img[py, px] = -2    
          
    return img, lines         
            

In [14]:
import cv2
import numpy as np
import pandas as pd
from pandas import DataFrame

# Teste de Erosão e Dilatação

'''

img = cv2.imread('input/t_16780_24831_16__c.png', 0)
img = cv2.bitwise_not(img)

kernel = np.ones((5, 5), np.uint8)
      
img_erosion = cv2.erode(img, kernel, iterations=1)
img_dilation = cv2.dilate(img, kernel, iterations=1)

  

cv2.imshow('Input', img)
cv2.imshow('Erosion', img_erosion)
cv2.imshow('Dilation', img_dilation)



  
cv2.waitKey(0)

'''



'''

img = cv2.imread('input/t_16780_24831_16__c.png', 0)
img = cv2.bitwise_not(img)
img1 = img.copy()

# Structuring Element
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
# Create an empty output image to hold values
thin = np.zeros(img.shape,dtype='uint8')

# Loop until erosion leads to an empty set
while (cv2.countNonZero(img1)!=0):
    # Erosion
    erode = cv2.erode(img1,kernel)
    # Opening on eroded image
    opening = cv2.morphologyEx(erode,cv2.MORPH_OPEN,kernel)
    # Subtract these two
    subset = erode - opening
    # Union of all previous sets
    thin = cv2.bitwise_or(subset,thin)
    # Set the eroded image for next iteration
    img1 = erode.copy()

df = DataFrame(thin)
df.to_csv("ex.csv")
cv2.imshow('Original',img)
cv2.imshow('Erodida',thin)
cv2.waitKey(0)


'''

'\n\nimg = cv2.imread(\'input/t_16780_24831_16__c.png\', 0)\nimg = cv2.bitwise_not(img)\nimg1 = img.copy()\n\n# Structuring Element\nkernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))\n# Create an empty output image to hold values\nthin = np.zeros(img.shape,dtype=\'uint8\')\n\n# Loop until erosion leads to an empty set\nwhile (cv2.countNonZero(img1)!=0):\n    # Erosion\n    erode = cv2.erode(img1,kernel)\n    # Opening on eroded image\n    opening = cv2.morphologyEx(erode,cv2.MORPH_OPEN,kernel)\n    # Subtract these two\n    subset = erode - opening\n    # Union of all previous sets\n    thin = cv2.bitwise_or(subset,thin)\n    # Set the eroded image for next iteration\n    img1 = erode.copy()\n\ndf = DataFrame(thin)\ndf.to_csv("ex.csv")\ncv2.imshow(\'Original\',img)\ncv2.imshow(\'Erodida\',thin)\ncv2.waitKey(0)\n\n\n'

In [15]:
# Teste da Transformada de Hough Probabilistica

'''
import cv2
import numpy as np

# Read image 
img = cv2.imread('input/t_16780_24831_16__c.png', cv2.IMREAD_COLOR) # road.png is the filename
# Convert the image to gray-scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Find the edges in the image using canny detector
edges = cv2.Canny(gray, 50, 200)
# Detect points that form a line

lines = cv2.HoughLinesP(edges, 1, np.pi/180, 40, minLineLength=10, maxLineGap=100)

# Draw lines on the image
for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
# Show result

cv2.imshow("Probabilistica", img)

cv2.waitKey(0)
'''


'\nimport cv2\nimport numpy as np\n\n# Read image \nimg = cv2.imread(\'input/t_16780_24831_16__c.png\', cv2.IMREAD_COLOR) # road.png is the filename\n# Convert the image to gray-scale\ngray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\n# Find the edges in the image using canny detector\nedges = cv2.Canny(gray, 50, 200)\n# Detect points that form a line\n\nlines = cv2.HoughLinesP(edges, 1, np.pi/180, 40, minLineLength=10, maxLineGap=100)\n\n# Draw lines on the image\nfor line in lines:\n    x1, y1, x2, y2 = line[0]\n    cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)\n# Show result\n\ncv2.imshow("Probabilistica", img)\n\ncv2.waitKey(0)\n'

In [16]:
# Teste da Transformada de Hough Padrão

'''
import cv2
import numpy as np

# Read image 
img = cv2.imread('input/t_16780_24831_16__c.png', cv2.IMREAD_COLOR) # road.png is the filename
# Convert the image to gray-scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Find the edges in the image using canny detector
edges = cv2.Canny(gray, 50, 200)
# Detect points that form a line

lines = cv2.HoughLines(edges,1,np.pi/180, 100)

for line in lines:
    for r,theta in line: 

        a = np.cos(theta) 
        b = np.sin(theta) 

        x0 = a*r 
        y0 = b*r 
        x1 = int(x0 + 1000*(-b)) 
        y1 = int(y0 + 1000*(a)) 
        x2 = int(x0 - 1000*(-b)) 
        y2 = int(y0 - 1000*(a)) 

        cv2.line(img,(x1,y1), (x2,y2), (0,0,255),2)
    # Show result

cv2.imshow("Padrao", img)

cv2.waitKey(0)
'''

'\nimport cv2\nimport numpy as np\n\n# Read image \nimg = cv2.imread(\'input/t_16780_24831_16__c.png\', cv2.IMREAD_COLOR) # road.png is the filename\n# Convert the image to gray-scale\ngray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\n# Find the edges in the image using canny detector\nedges = cv2.Canny(gray, 50, 200)\n# Detect points that form a line\n\nlines = cv2.HoughLines(edges,1,np.pi/180, 100)\n\nfor line in lines:\n    for r,theta in line: \n\n        a = np.cos(theta) \n        b = np.sin(theta) \n\n        x0 = a*r \n        y0 = b*r \n        x1 = int(x0 + 1000*(-b)) \n        y1 = int(y0 + 1000*(a)) \n        x2 = int(x0 - 1000*(-b)) \n        y2 = int(y0 - 1000*(a)) \n\n        cv2.line(img,(x1,y1), (x2,y2), (0,0,255),2)\n    # Show result\n\ncv2.imshow("Padrao", img)\n\ncv2.waitKey(0)\n'

In [17]:
# Teste de convolução da imagem.

'''

def processImage(image): 
  image = cv2.imread(image) 
  image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY) 
  return image


def convolve2D(image, kernel, padding=0, strides=1):
    # Cross Correlation
    kernel = np.flipud(np.fliplr(kernel))

    # Gather Shapes of Kernel + Image + Padding
    xKernShape = kernel.shape[0]
    yKernShape = kernel.shape[1]
    xImgShape = image.shape[0]
    yImgShape = image.shape[1]

    # Shape of Output Convolution
    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)
    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))
        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
        print(imagePadded)
    else:
        imagePadded = image

    # Iterate through image
    for y in range(image.shape[1]):
        # Exit Convolution
        if y > image.shape[1] - yKernShape:
            break
        # Only Convolve if y has gone down by the specified Strides
        if y % strides == 0:
            for x in range(image.shape[0]):
                # Go to next row once kernel is out of bounds
                if x > image.shape[0] - xKernShape:
                    break
                try:
                    # Only Convolve if x has moved by the specified Strides
                    if x % strides == 0:
                        output[x, y] = (kernel * imagePadded[x: x + xKernShape, y: y + yKernShape]).sum()
                except:
                    break

    return output

# Grayscale Image
image = processImage('input/t_16780_24831_16__c.png')

# Edge Detection Kernel
kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])

# Convolve and Save Output
output = convolve2D(image, kernel, padding=2)
cv2.imwrite('2DConvolved.jpg', output)

'''

"\n\ndef processImage(image): \n  image = cv2.imread(image) \n  image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY) \n  return image\n\n\ndef convolve2D(image, kernel, padding=0, strides=1):\n    # Cross Correlation\n    kernel = np.flipud(np.fliplr(kernel))\n\n    # Gather Shapes of Kernel + Image + Padding\n    xKernShape = kernel.shape[0]\n    yKernShape = kernel.shape[1]\n    xImgShape = image.shape[0]\n    yImgShape = image.shape[1]\n\n    # Shape of Output Convolution\n    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)\n    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)\n    output = np.zeros((xOutput, yOutput))\n\n    # Apply Equal Padding to All Sides\n    if padding != 0:\n        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))\n        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image\n        print(imagePadded)\n    else:\n        imagePadded = image\n\n    #

In [18]:
from skimage.morphology import skeletonize, thin
from skimage import data
import matplotlib.pyplot as plt
from skimage.util import invert

import cv2
from os import listdir
from os.path import isfile, join

# Testando Skeletonize
'''
path_in = 'input'
path = 'output'

for file in listdir(path):
    if isfile(join(path, file)):

        # Invert the horse image
        f = file[:16] + '.png'
        img = cv2.imread(join(path_in, f), cv2.IMREAD_COLOR)
        
        print(img.shape)
        
        #Cortando a imagem
        img = img[0:256, 256:512]
        
        image = invert(img)
        

        # perform skeletonization
        skeleton = skeletonize(image)
        

        # display results
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 8),
                                sharex=True, sharey=True)

        ax = axes.ravel()

        ax[0].imshow(image, cmap=plt.cm.gray)
        ax[0].axis('off')
        ax[0].set_title('original', fontsize=20)

        ax[1].imshow(skeleton, cmap=plt.cm.gray)
        ax[1].axis('off')
        ax[1].set_title('skeleton', fontsize=20)
        

        fig.tight_layout()
        plt.show()
        
        
'''

"\npath_in = 'input'\npath = 'output'\n\nfor file in listdir(path):\n    if isfile(join(path, file)):\n\n        # Invert the horse image\n        f = file[:16] + '.png'\n        img = cv2.imread(join(path_in, f), cv2.IMREAD_COLOR)\n        \n        print(img.shape)\n        \n        #Cortando a imagem\n        img = img[0:256, 256:512]\n        \n        image = invert(img)\n        \n\n        # perform skeletonization\n        skeleton = skeletonize(image)\n        \n\n        # display results\n        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 8),\n                                sharex=True, sharey=True)\n\n        ax = axes.ravel()\n\n        ax[0].imshow(image, cmap=plt.cm.gray)\n        ax[0].axis('off')\n        ax[0].set_title('original', fontsize=20)\n\n        ax[1].imshow(skeleton, cmap=plt.cm.gray)\n        ax[1].axis('off')\n        ax[1].set_title('skeleton', fontsize=20)\n        \n\n        fig.tight_layout()\n        plt.show()\n        \n        \n"

In [19]:
# Script para cortar as imagens
'''
import cv2
img = cv2.imread("input/root/t_16780_24831_16.png")
crop_img = img[0:256, 256:512]
cv2.imshow("cropped", crop_img)
cv2.waitKey(0)

'''

'\nimport cv2\nimg = cv2.imread("input/root/t_16780_24831_16.png")\ncrop_img = img[0:256, 256:512]\ncv2.imshow("cropped", crop_img)\ncv2.waitKey(0)\n\n'

In [20]:
from skimage.morphology import skeletonize, thin
from skimage import data
import matplotlib.pyplot as plt
from skimage.util import invert
import numpy as np

import cv2
from os import listdir
from os.path import isfile, join

# Testando Skeletonize


path = 'input'
path_out = 'output'

for file in listdir(path):
    if isfile(join(path, file)):

        f = file
        img = cv2.imread(join(path, f), cv2.IMREAD_COLOR)

        
        #Cortando a imagem
        img = img[0:256, 256:512, 0:1]
        
        image = invert(img)

        # perform skeletonization
        image = skeletonize(image)

        image = invert(image)

        img = image.copy().astype(np.int32)

        img[img[:, :,0] == 255] = -1
        
        i ,lines = findLine(img)

        img[img[:, :,0] != -1] = 0
        img[img[:, :,0] == -1] = 255
        
        j = 1

        linesCoord = []
        for line in lines:
            l = []
            for c in line:
                val, px, py = c

                x = int(file[2:7])
                y = int(file[8:13])
                z = int(file[14:16])

                coord = pixToCoord(x, y, z, 256, 256, px, py)

                l.append(coord)
            
            linesCoord.append(l)
        
        f = file[:16] + '.csv'
        df = DataFrame(linesCoord)

        df.to_csv(join(path_out, f))

        #cv2.imshow("Imagens Skeletonize", image)
        #cv2.waitKey(0)

    

In [21]:
'''
import cv2
import numpy as np

imgDf = pd.read_csv('output/t_16780_24831_16.csv')

        
rows, cols = imgDf.shape
for i in range(rows):
    for j in range(cols):
        if imgDf.iloc[i, j] == 255:
            imgDf.iloc[i, j] = -1


imgNp = np.array(imgDf)

img, lines = findLine(imgNp)


img = cv2.imread('input/t_16780_24831_16.png', cv2.IMREAD_COLOR)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 200)

for line in lines:
    for i in range(0, len(line)):
        
        if i+1 < len(line):
            start = line[i]
            end = line[i+1]

            startV, startPx, startPy = start
            endV, endPx, endPy = end

            cv2.line(img, (startPx, startPy), (endPx, endPy), (255, 0, 0), 2)

cv2.imshow("Result Image", img)
cv2.waitKey(0)
'''

'\nimport cv2\nimport numpy as np\n\nimgDf = pd.read_csv(\'output/t_16780_24831_16.csv\')\n\n        \nrows, cols = imgDf.shape\nfor i in range(rows):\n    for j in range(cols):\n        if imgDf.iloc[i, j] == 255:\n            imgDf.iloc[i, j] = -1\n\n\nimgNp = np.array(imgDf)\n\nimg, lines = findLine(imgNp)\n\n\nimg = cv2.imread(\'input/t_16780_24831_16.png\', cv2.IMREAD_COLOR)\ngray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\nedges = cv2.Canny(gray, 50, 200)\n\nfor line in lines:\n    for i in range(0, len(line)):\n        \n        if i+1 < len(line):\n            start = line[i]\n            end = line[i+1]\n\n            startV, startPx, startPy = start\n            endV, endPx, endPy = end\n\n            cv2.line(img, (startPx, startPy), (endPx, endPy), (255, 0, 0), 2)\n\ncv2.imshow("Result Image", img)\ncv2.waitKey(0)\n'

In [22]:
'''

import cv2
import numpy as np

path = 'output'
path_in = 'input'
listCoord = []

for file in listdir(path):
    if isfile(join(path, file)):
        

        imgDf = pd.read_csv(join(path, file))

        
        rows, cols = imgDf.shape
        for i in range(rows):
            for j in range(cols):
                if imgDf.iloc[i, j] == 255:
                    imgDf.iloc[i, j] = -1
        

        imgNp = np.array(imgDf)

        img, lines = findLine(imgNp)

        f = file[:16] + '.png'
        print(join(path_in, f))
        img = cv2.imread(join(path_in, f), cv2.IMREAD_COLOR)


        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 200)
        
        for line in lines:
            for i in range(0, len(line)):
                
                if i+1 < len(line):
                    start = line[i]
                    end = line[i+1]

                    startV, startPx, startPy = start
                    endV, endPx, endPy = end

                    cv2.line(img, (startPx, startPy), (endPx, endPy), (255, 0, 0), 3)

        cv2.imshow("Result Image", img)
        cv2.waitKey(0)


'''

'\n\nimport cv2\nimport numpy as np\n\npath = \'output\'\npath_in = \'input\'\nlistCoord = []\n\nfor file in listdir(path):\n    if isfile(join(path, file)):\n        \n\n        imgDf = pd.read_csv(join(path, file))\n\n        \n        rows, cols = imgDf.shape\n        for i in range(rows):\n            for j in range(cols):\n                if imgDf.iloc[i, j] == 255:\n                    imgDf.iloc[i, j] = -1\n        \n\n        imgNp = np.array(imgDf)\n\n        img, lines = findLine(imgNp)\n\n        f = file[:16] + \'.png\'\n        print(join(path_in, f))\n        img = cv2.imread(join(path_in, f), cv2.IMREAD_COLOR)\n\n\n        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\n        edges = cv2.Canny(gray, 50, 200)\n        \n        for line in lines:\n            for i in range(0, len(line)):\n                \n                if i+1 < len(line):\n                    start = line[i]\n                    end = line[i+1]\n\n                    startV, startPx, startPy = start\n